In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, XSD, RDFS, SOSA, ConjunctiveGraph
from rdflib.plugins.stores import sparqlstore
from rdflib.graph import DATASET_DEFAULT_GRAPH_ID as default
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST
import requests
import json
from shapely.geometry import shape
import os
import xarray as xr
os.chdir("/workspaces/aqqa-kg-creation-dev/")

In [36]:
# loading ref raster cells
ref_raster_geojson = "/mnt/data/processed/ref_raster.geojson"
with open(ref_raster_geojson, "r") as f:
    geojson_data = json.load(f)

features = geojson_data.get("features", [])
geometries = [feature.get("geometry") for feature in features]
indexes = [feature.get("properties")["index"] for feature in features]
shapely_geometries = [shape(geometry).wkt for geometry in geometries]

In [37]:
# Define the namespaces
aqqa = Namespace("http://example.com/ontologies/aqqa#")
geo = Namespace("http://www.opengis.net/ont/geosparql#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
sf = Namespace("http://www.opengis.net/ont/sf#")

# Create a graph
g = Graph()
g.bind("aqqa", aqqa)

In [38]:
# reading geometries and Features of interest into RDF graph
for i, (index, geom) in enumerate(zip(indexes, shapely_geometries)):
    
    ent_geom_cell = URIRef(aqqa[f"GeomCell_{index[0]}_{index[1]}"])   
    ent_cell = URIRef(aqqa[f"Cell_{index[0]}_{index[1]}"])   
    ent_hasID = URIRef(aqqa["hasID"])

    g.add((ent_cell, RDF.type, SOSA.FeatureOfInterest))
    g.add((ent_cell, ent_hasID, Literal(i)))
    g.add((ent_cell, geo.hasGeometry, ent_geom_cell))
    
    g.add((ent_geom_cell, RDF.type, sf.Geometry))
    g.add((ent_geom_cell, geo.asWKT, Literal(geom, datatype=geo.wktLiteral)))

In [39]:
# Execute Query 1 (get variable, time and result of all observations)
query1 = """    
        SELECT ?s ?p ?o
        WHERE {
            ?s ?p ?o
        }
        LIMIT 10
       """

results1 = g.query(query1)
print("Query 1 Results:")
for row in results1:
    print(f"{row['s']} {row['p']} {row['o']} ")

Query 1 Results:
http://example.com/ontologies/aqqa#GeomCell_38_54 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.opengis.net/ont/sf#Geometry 
http://example.com/ontologies/aqqa#Cell_17_46 http://www.opengis.net/ont/geosparql#hasGeometry http://example.com/ontologies/aqqa#GeomCell_17_46 
http://example.com/ontologies/aqqa#GeomCell_21_62 http://www.opengis.net/ont/geosparql#asWKT POLYGON ((15.15 47.95, 15.25 47.95, 15.25 48.05, 15.15 48.05, 15.15 47.95)) 
http://example.com/ontologies/aqqa#Cell_3_47 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/ns/sosa/FeatureOfInterest 
http://example.com/ontologies/aqqa#Cell_35_38 http://www.opengis.net/ont/geosparql#hasGeometry http://example.com/ontologies/aqqa#GeomCell_35_38 
http://example.com/ontologies/aqqa#GeomCell_41_59 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.opengis.net/ont/sf#Geometry 
http://example.com/ontologies/aqqa#GeomCell_14_74 http://www.opengis.net/ont/geosparql#asWKT POLYGON ((16.3

In [40]:
g.serialize(destination="/mnt/data/processed/RDF/CAMS/CAMS_ref_grid.ttl")

<Graph identifier=N08363365a46e4ef0a1b538ba3ec84e53 (<class 'rdflib.graph.Graph'>)>